# Lab 12 (Apr 6): Linear Programming

In [ ]:
import numpy as np
from numpy import linalg as LA
import cvxpy as cp
import matplotlib.pyplot as plt

Motivation: A few examples of geometric problems can be formulated, and hence solved, as linear programs. 

Method: 
* Using CVXPY to solve linear programming (I will use this one only)
* you can use scipy.optimize.linprog as well

## Inscribed circle to a polytope

Warm-up example:

Find the largest $\ell_2$-ball contained in a polytope $\mathcal{C} = \{ x\in\mathbb{R}^2:x_1+x_2\leq1, x_1\geq0, x_2\geq0\}$, (see Figure below). 

By simple Pythagorean theorem argument, we know that the largest ball is centered at $(r,r)$ with radius $r$, where $r=\frac{1}{2+\sqrt{2}}$. 

It can be done by solving a linear programming problem as well. Moreover, we can use linear programming to find the largest inscribed circle to any polytope. See Example 20.5 in the textbook.

In [ ]:
x = np.linspace(0,1,100)
y = 1-x
fig, ax = plt.subplots() 
ax.plot(x, y, '-k')
ax.axhline(y=0,xmin=0, xmax=1,color='k')
ax.axvline(x=0,ymin=0, ymax=1,color='k')
r = 1/(2+np.sqrt(2))
circle1 = plt.Circle((r, r), r, color='r')
ax.add_patch(circle1)
ax.plot(r, r, '-o',color='b',markersize=3)
plt.axis('equal')
plt.show()

Finding the largest $\ell_p$-ball contained in a polytope $\mathcal{C} = \{ x\in\mathbb{R}^d:\langle a^{(1)},x\rangle\leq b_1,\dots, \langle a^{(n)},x\rangle\leq b_n\}$ amounts to solving the following linear program: $$ \underset{c\in\mathbb{R}^d,r\in\mathbb{R}}{\mathrm{maximize}} \;r \quad \mbox{ subject to } \langle a^{(i)},c\rangle + r\|a^{(i)}\|_q \leq b_i, i=1,\dots,n,$$ where $q:=p/(p-1)\in[1,\infty]$ is the conjugate exponent of $p\in[1,\infty]$.

In our trivial example, $p=q=2$ and $d=2$. Polytope $\mathcal{C}$ is constructed with
$$ a^{(1)} = [1,1]^\top$$
$$ a^{(2)} = [-1,0]^\top$$
$$ a^{(3)} = [0,-1]^\top$$
$$ b = [1,0,0]^\top.$$

In [ ]:
# Example of the d-simplex, first in the euclidean plane (p=2,d=2)
# A picture reveals that the radius of the inscribed circle is 1/(2+sqrt(2))
d = 2                       # ambient dimension
p = 2                       # index of the \ell_p-norm
q = p/(p-1)                 # conjugate index
A = np.row_stack( (np.ones((1,d)) , -np.eye(d)) )
b = np.row_stack( (1, np.zeros((d,1)) ) )
row_norms = np.zeros((d+1,1))
for i in range(d+1):
    row_norms[i] = LA.norm(A[i,:],q)
center = cp.Variable((d,1))
radius = cp.Variable(1)
objective = cp.Maximize(radius)
constraints = [A@center + radius*row_norms <= b]
inscribed_circle = cp.Problem(objective,constraints)
inscribed_circle.solve()
radius = radius.value[0]
print('In the euclidean plane, the computed inscribed radius is {:.3f}, which indeed equals 1/(2+sqrt(2))={:.3f}'
      .format(radius,1/(2+np.sqrt(2))))

Now we move to the general case (Exercise 20.4)

When $\mathcal{C}$ is the d-simplex, it is possible to guess a formula providing the value of the radius of the inscribed $\ell_p$-ball as a function of $d$ and $p$.

In [ ]:
## Compute the values of the inscribed radius as a function of d when p=2, p=Inf, and p=1

d_max = 10
P = [1,2,np.inf]
Q = [np.inf,2,1]
for k in range(3):
    # This loop controls the values of p and q
    
    for d in range(1,d_max+1):
        # This loop controls the dimension d
        
        for i in range(d+1):
            row_norms[i] = LA.norm(A[i,:],q)
        
        # linear programming goes here with pre-defined p,q,d
        
    print('\nFor p={}, the radius of the inscribed radii when d goes from 1 to {} are:\n{}'
          .format(p,d_max,np.round(radius,4)))